Ситуация: на платформе появилось много плохих комментариев. От продакт-менеджера поступила задача - избавиться от плохих комментариев.
Переход от продуктовой задачи к ML задаче:
Разработать бинарный классификатор, который на вход будет принимать текст комментария, на выходе выдавать 2 класса - хороший/плохой комментарий.
Метрики и ограничения:
Задача - максимизировать количество удаленных негативных комментариев. В идеальном случае мы должны удалять их все, но мы не хотим удалять хорошие комментарии,
чтобы пользователи не обижались
Ограничение: в 1 из 20 случаев мы имеем право ошибиться, то есть удалить хороший комментарий вместо плохого.
2 метрики:
recall -  соотношение найденных плохих комментариев
(если эта метрика = 1, значит мы нашли всеплохие комментарии)
precision - вероятность того, что если мы сказали, что комментарий является реально плохим, то он им реально является
(поставим precision >0.95)
В итоге задача:
> Разработать бинарный классификатор
> Максимизировать recall
> precision > 0.95
Следующий этап: Составление датасета
Можно разметить данные самому, но это очень дорого, поэтому попробуем найти готовый датасет.
Прям так и пишем в гугл: "датасет токсичных комментариев"
находим https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')  # Скачиваем стоп-слова
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Анализ данных (EDA)

In [2]:
df = pd.read_csv("./data/labeled.csv", sep=",")

In [3]:
df.shape

(14412, 2)

In [4]:
df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


видим, что в столбце toxic данные типа float. переведем их в тип int.

In [5]:
df["toxic"] = df["toxic"].apply(int)

In [6]:
df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


посмотрим распределение данных.

In [7]:
df["toxic"].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

посмотрим, что подразумевается под токсичными комментариями в данном датасете.

In [8]:
for c in df[df["toxic"]==1]["comment"].head(5):
    print(c)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [9]:
for c in df[df["toxic"]==0]["comment"].head(5):
    print(c)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

видим, что разметка данных не очень хорошая, так как даже в положительные комментарии попали достаточно негативные высказывания. но работать будем с тем, что имеется.

В реальности для тестового датасета нужно использовать данные с таким же распределением, что и в продакшн системе, но у нас нет на самом деле никакго сервиса, поэтому возьмем 500 сэймплов из нашего датасета как тестовые и предположим, что распределение в них совпадает с нашей придуманной системой.
используем функцию train_test_split из библиотеки scikit-learn

In [10]:
train_df, test_df = train_test_split(df, test_size=500)

проверим, что у нас действительно 500 samples на тест

In [11]:
test_df.shape

(500, 2)

проверим распределение в test и в train, удостоверимся, что оно примерно одинаковое.

In [12]:
test_df["toxic"].value_counts()

toxic
0    312
1    188
Name: count, dtype: int64

In [13]:
train_df["toxic"].value_counts()

toxic
0    9274
1    4638
Name: count, dtype: int64

и там и там распределение 1 к 2.

Буду испозльзовать самую простую модель - логистическую регрессию из библиотеки scikit-learn.  
Перед тем как подавать данные на вход, надо из текста сделать численные векторы, потому что сама модель принимает на вход вещественные векторы.

## Предобработка текста

Этапы:  
- Разбить текст на токены
- Удалить токены, не несущие смысловой информации (знаки пунктуации, стоп слова)  
- К каждому токену применить стемминг (stemming)(удалить окончания, привести все к нижнему регистру)

In [19]:
print(nltk.data.find('tokenizers/punkt'))

C:\Users\User\AppData\Roaming\nltk_data\tokenizers\punkt


In [31]:
sentence_example = df.iloc[1]["comment"]

tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")

stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [32]:
print(f"Исходный текст: {sentence_example}")
print("--------------------")
print(f"Токены: {tokens}")
print("--------------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("--------------------")
print(f"Токены без пунктуации и стоп-слов: {tokens_without_stop_words_and_punctuation}")
print("--------------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("--------------------")

Исходный текст: Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

--------------------
Токены: ['Хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'Если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
--------------------
Токены без пунктуации: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'а', 'у', 'хохлов', 'еще', 'хуже', 'Если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']
--------------------
Токены без пунктуации и стоп-слов: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'хуже', 'Если', 'хохлов', 'кисель', 'придумал']
--------------------
Токены после стемминга: ['хохл', 'эт', 'отдушин', 'затюкан', 'россиянин', 'мол', 'вон', 'хохл', 'хуж', 'есл', 'хохл', 'кисел', 'придума']
--------------------


напишем функцию, чтобы обработать все значения

In [33]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

проверим ее на примере

In [34]:
tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']